In [1]:
import numpy as np
import scipy as sc
import pandas as pd
from scipy import optimize
from copy import deepcopy
import itertools

# Single Table Many Attribute

In [10]:
# 7. > 的误差：通过增加一个 sys.maxsize 区间来解决【已解决】
# 8. 表中的数据应该都是 >= 0 的？query中也不会产生小于0的condition吧？目前代码没有处理 < 0 的逻辑
# 6. 每次都取左端点的值，比随机在区间中取值的误差要小（结合op_to_variable实现）【已解决】

In [11]:
import datasets
import argparse
import estimators as estimators_lib
import sys
import numpy as np
import pandas as pd
from scipy import optimize
from copy import deepcopy
import itertools

In [12]:
def Oracle(table, query):
    cols, idxs, ops, vals = query
    oracle_est = estimators_lib.Oracle(table)
    return oracle_est.Query(cols, ops, vals)


def cal_true_card(query, table):
    cols, idxs, ops, vals = query
    ops = np.array(ops)
    probs = Oracle(table, (cols, idxs, ops, vals))
    return probs


def GenerateQuery(table, min_num_filters, max_num_filters, rng, dataset):
    """Generate a random query."""
    num_filters = rng.randint(max_num_filters - 1, max_num_filters)
    cols, idxs, ops, vals = SampleTupleThenRandom(table, num_filters, rng, dataset)
    sel = cal_true_card((cols, idxs, ops, vals), table) / float(table.cardinality)
    return cols, idxs, ops, vals, sel


def SampleTupleThenRandom(table, num_filters, rng, dataset):
    vals = []
    new_table = table.data
    s = new_table.iloc[rng.randint(0, new_table.shape[0])]
    vals = s.values
    if dataset in ["dmv", "dmv-tiny", "order_line"]:
        vals[6] = vals[6].to_datetime64()
    elif dataset in ["orders1", "orders"]:
        vals[4] = vals[4].to_datetime64()
    elif dataset == "lineitem":
        vals[10] = vals[10].to_datetime64()
        vals[11] = vals[11].to_datetime64()
        vals[12] = vals[12].to_datetime64()
    idxs = rng.choice(len(table.columns), replace=False, size=num_filters)
    cols = np.take(table.columns, idxs)
    # If dom size >= 10, okay to place a range filter.
    # Otherwise, low domain size columns should be queried with equality.
    ops = rng.choice(["<=", ">=", "="], size=num_filters)
    ops_all_eqs = ["="] * num_filters
    sensible_to_do_range = [c.DistributionSize() >= 10 for c in cols]
    ops = np.where(sensible_to_do_range, ops, ops_all_eqs)
    # if num_filters == len(table.columns):
    #     return table.columns,np.arange(len(table.columns)), ops, vals
    vals = vals[idxs]
    op_a = []
    val_a = []
    for i in range(len(vals)):
        val_a.append([vals[i]])
        op_a.append([ops[i]])
    return cols, idxs, pd.DataFrame(op_a).values, pd.DataFrame(val_a).values

In [13]:
parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str, default="wine", help="Dataset.")
parser.add_argument("--device", type=str, default="cuda:0", help="Device")
parser.add_argument("--query-size", type=int, default=1000, help="query size")
parser.add_argument("--reload", type=bool, default=False, help="query size")
parser.add_argument("--num-conditions", type=int, default=1000, help="num of conditions")
parser.add_argument("--lr", type=float, default=5e-3, help="learning rate")

# args = parser.parse_args()()
args, unknown = parser.parse_known_args()

In [14]:
table = datasets.LoadDataset(args.dataset + ".csv", args.dataset, type_casts={})

load dataset wine.csv done
(6497, 13)
(6497, 13)
0
1
2
3
4
5
6
7
8
9
10
11
12
0 106
1 187
2 89
3 316
4 214
5 135
6 276
7 975
8 108
9 111
10 111
11 7
12 2


In [15]:
args.dataset = "wine"
# args.device = 'cpu'
args.query_size = 100
# args.lr = 0.0002
args.num_conditions = 2
args.reload = False

In [16]:
if not args.reload:
    print("Begin Generating Queries ...")
    rng = np.random.RandomState(1234)
    query_set = [
        GenerateQuery(table, 2, args.num_conditions + 1, rng, args.dataset)
        for i in range(args.query_size)
    ]
    print("Complete Generating Queries ...")

Begin Generating Queries ...
Complete Generating Queries ...


In [17]:
query_set

[(array([Column(10, distribution_size=111),
         Column(0, distribution_size=106)], dtype=object),
  array([10,  0]),
  array([['<='],
         ['=']], dtype=object),
  array([[12.3],
         [ 6.5]]),
  0.03340003078343851),
 (array([Column(9, distribution_size=111), Column(7, distribution_size=975)],
        dtype=object),
  array([9, 7]),
  array([['='],
         ['=']], dtype=object),
  array([[0.6   ],
         [0.9994]]),
  0.00015391719255040788),
 (array([Column(8, distribution_size=108), Column(5, distribution_size=135)],
        dtype=object),
  array([8, 5]),
  array([['='],
         ['<=']], dtype=object),
  array([[ 3.04],
         [47.  ]]),
  0.013544712944435893),
 (array([Column(8, distribution_size=108), Column(2, distribution_size=89)],
        dtype=object),
  array([8, 2]),
  array([['>='],
         ['<=']], dtype=object),
  array([[3.09],
         [0.36]]),
  0.5704171155918116),
 (array([Column(3, distribution_size=316), Column(5, distribution_size=135)],
  

In [18]:
table_size = table.data.shape
n_row = table_size[0]
n_column = table_size[1]

In [19]:
def dictionary_column_interval(table_size, query_set):
    n_column = table_size[1]
    column_interval = {}
    for i in range(n_column):
        column_interval[i] = set([0, sys.maxsize])
    for query in query_set:
        col_idxs = query[1]
        # ops = query[2]
        vals = query[3]
        # sel = query[4]
        for i in range(len(col_idxs)):
            column_interval[col_idxs[i]].add(vals[i][0])
    for k, v in column_interval.items():
        column_interval[k] = sorted(list(v))
    return column_interval

In [20]:
column_to_interval = dictionary_column_interval(table_size, query_set)

In [21]:
column_to_interval

{0: [0,
  5.1,
  6.3,
  6.4,
  6.5,
  6.6,
  6.7,
  6.8,
  6.9,
  7.1,
  7.2,
  7.3,
  7.4,
  7.5,
  8.4,
  8.9,
  9223372036854775807],
 1: [0,
  0.2,
  0.22,
  0.24,
  0.25,
  0.26,
  0.29,
  0.3,
  0.34,
  0.49,
  0.69,
  0.74,
  9223372036854775807],
 2: [0,
  0.08,
  0.1,
  0.15,
  0.27,
  0.33,
  0.35,
  0.36,
  0.37,
  0.42,
  9223372036854775807],
 3: [0, 1.05, 1.2, 1.4, 1.8, 2.0, 2.3, 5.0, 8.9, 9223372036854775807],
 4: [0,
  0.026,
  0.028,
  0.029,
  0.033,
  0.034,
  0.041,
  0.042,
  0.043,
  0.044,
  0.045,
  0.052,
  0.054,
  0.055,
  0.057,
  0.062,
  0.09,
  0.092,
  0.105,
  0.244,
  9223372036854775807],
 5: [0,
  7.0,
  9.0,
  17.0,
  21.0,
  24.0,
  25.0,
  28.0,
  31.0,
  37.0,
  39.0,
  47.0,
  9223372036854775807],
 6: [0,
  11.0,
  40.0,
  60.0,
  80.0,
  87.0,
  93.0,
  110.0,
  111.0,
  121.0,
  140.0,
  152.0,
  158.0,
  171.0,
  183.0,
  190.0,
  9223372036854775807],
 7: [0,
  0.98959,
  0.99014,
  0.99226,
  0.99228,
  0.9929,
  0.99381,
  0.9945,
  0.995

In [22]:
def dictionary_column_variable(column_to_interval):
    total_variables = 0
    column_to_variable = {}
    for k, v in column_to_interval.items():
        count = len(v)
        column_to_variable[k] = [total_variables + i for i in range(count)]
        total_variables += count
    return total_variables, column_to_variable

In [23]:
total_variables, column_to_variable = dictionary_column_variable(column_to_interval)

In [24]:
total_variables

178

In [25]:
column_to_variable

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 1: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 2: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
 3: [41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
 4: [51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71],
 5: [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84],
 6: [85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101],
 7: [102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 8: [117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138],
 9: [139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153],
 10: [154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166],
 11: [167, 168, 169, 170, 171, 172, 173],
 12: [174, 175, 176, 177]}

In [26]:
def dictionary_variable_interval(column_to_interval, column_to_variable):
    variable_to_interval = {}
    for column, variable in column_to_variable.items():
        for i in range(len(variable)):
            variable_to_interval[variable[i]] = column_to_interval[column][i]
    return variable_to_interval

In [27]:
variable_to_interval = dictionary_variable_interval(column_to_interval, column_to_variable)

In [28]:
variable_to_interval

{0: 0,
 1: 5.1,
 2: 6.3,
 3: 6.4,
 4: 6.5,
 5: 6.6,
 6: 6.7,
 7: 6.8,
 8: 6.9,
 9: 7.1,
 10: 7.2,
 11: 7.3,
 12: 7.4,
 13: 7.5,
 14: 8.4,
 15: 8.9,
 16: 9223372036854775807,
 17: 0,
 18: 0.2,
 19: 0.22,
 20: 0.24,
 21: 0.25,
 22: 0.26,
 23: 0.29,
 24: 0.3,
 25: 0.34,
 26: 0.49,
 27: 0.69,
 28: 0.74,
 29: 9223372036854775807,
 30: 0,
 31: 0.08,
 32: 0.1,
 33: 0.15,
 34: 0.27,
 35: 0.33,
 36: 0.35,
 37: 0.36,
 38: 0.37,
 39: 0.42,
 40: 9223372036854775807,
 41: 0,
 42: 1.05,
 43: 1.2,
 44: 1.4,
 45: 1.8,
 46: 2.0,
 47: 2.3,
 48: 5.0,
 49: 8.9,
 50: 9223372036854775807,
 51: 0,
 52: 0.026,
 53: 0.028,
 54: 0.029,
 55: 0.033,
 56: 0.034,
 57: 0.041,
 58: 0.042,
 59: 0.043,
 60: 0.044,
 61: 0.045,
 62: 0.052,
 63: 0.054,
 64: 0.055,
 65: 0.057,
 66: 0.062,
 67: 0.09,
 68: 0.092,
 69: 0.105,
 70: 0.244,
 71: 9223372036854775807,
 72: 0,
 73: 7.0,
 74: 9.0,
 75: 17.0,
 76: 21.0,
 77: 24.0,
 78: 25.0,
 79: 28.0,
 80: 31.0,
 81: 37.0,
 82: 39.0,
 83: 47.0,
 84: 9223372036854775807,
 85: 0,
 86:

In [29]:
def op_to_variables(column_to_variable, column, x_index, op):
    variable = np.array(column_to_variable[column])
    if op == ">":
        return list(variable[x_index + 1 :])
    elif op == ">=":
        return list(variable[x_index:])
    elif op == "=":
        return [variable[x_index]]
    elif op == "<":
        return list(variable[:x_index])
    elif op == "<=":
        return list(variable[: x_index + 1])

In [30]:
def dictionary_query_to_variable(query_set, column_to_interval, column_to_variable):
    query_to_variable = {}
    for i in range(len(query_set)):
        col_idxs = query_set[i][1]
        ops = query_set[i][2]
        vals = query_set[i][3]
        # sel = query_set[i][4]
        query_to_variable[i] = []
        for j in range(len(col_idxs)):
            column = col_idxs[j]
            x_index = column_to_interval[column].index(vals[j][0])
            query_to_variable[i].append(
                op_to_variables(column_to_variable, column, x_index, ops[j][0])
            )
    return query_to_variable

In [31]:
query_to_variable = dictionary_query_to_variable(query_set, column_to_interval, column_to_variable)

In [32]:
query_to_variable

{0: [[154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165], [4]],
 1: [[147], [115]],
 2: [[120], [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]],
 3: [[122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138],
  [30, 31, 32, 33, 34, 35, 36, 37]],
 4: [[43, 44, 45, 46, 47, 48, 49, 50], [78]],
 5: [[24], [131, 132, 133, 134, 135, 136, 137, 138]],
 6: [[48, 49, 50], [75]],
 7: [[171], [47]],
 8: [[176], [102, 103, 104, 105, 106, 107]],
 9: [[171], [85, 86, 87, 88, 89, 90, 91, 92, 93, 94]],
 10: [[85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100],
  [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]],
 11: [[170], [30, 31, 32, 33]],
 12: [[108, 109, 110, 111, 112, 113, 114, 115, 116],
  [124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138]],
 13: [[159], [15]],
 14: [[73],
  [117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,

In [33]:
# 同时创建 x: other x 的字典，根据query找到每个x绑定的另外的x，用来generate数据时候每列同时匹配地插入数据

In [34]:
# col: query 暂时用不到

In [35]:
def seperate_column_variable(column_to_variable, query_to_variable):
    used_variable = set()
    for k, v in query_to_variable.items():
        for x in v:
            used_variable.update(set(x))
    column_used_variable = deepcopy(column_to_variable)
    for k, v in column_used_variable.items():
        column_used_variable[k] = sorted(list(set(v) & used_variable))
    column_unused_variable = deepcopy(column_to_variable)
    for k, v in column_unused_variable.items():
        column_unused_variable[k] = sorted(list(set(v) - used_variable))
    return used_variable, column_used_variable, column_unused_variable

In [36]:
used_x, col_used_x, col_unused_x = seperate_column_variable(column_to_variable, query_to_variable)

In [37]:
used_x

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 169,
 170,
 171,
 172,
 175,
 176}

In [38]:
col_used_x

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 1: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 2: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
 3: [41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
 4: [51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71],
 5: [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84],
 6: [85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101],
 7: [102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 8: [117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138],
 9: [139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153],
 10: [154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166],
 11: [168, 169, 170, 171, 172],
 12: [175, 176]}

In [39]:
col_unused_x

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [167, 173],
 12: [174, 177]}

In [40]:
# 生成整数规划模型，并求解

In [41]:
def table_constraints(column_to_variable):
    table_constraints_list = []
    for value in column_to_variable.values():

        def column_constraints(x, value=value):
            length = 0
            for idx in value:
                length += x[idx]
            return length - n_row

        table_constraints_list.append(column_constraints)
    return table_constraints_list

In [42]:
table_constraints_list = table_constraints(column_to_variable)

In [43]:
table_constraints_list

[<function __main__.table_constraints.<locals>.column_constraints(x, value=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])>,
 <function __main__.table_constraints.<locals>.column_constraints(x, value=[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])>,
 <function __main__.table_constraints.<locals>.column_constraints(x, value=[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])>,
 <function __main__.table_constraints.<locals>.column_constraints(x, value=[41, 42, 43, 44, 45, 46, 47, 48, 49, 50])>,
 <function __main__.table_constraints.<locals>.column_constraints(x, value=[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71])>,
 <function __main__.table_constraints.<locals>.column_constraints(x, value=[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84])>,
 <function __main__.table_constraints.<locals>.column_constraints(x, value=[85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101])>,
 <function __main__.table_constraints.<locals>.

In [44]:
def query_constraints(query_set, query_to_variable):
    query_constraints_list = []
    for key, values in query_to_variable.items():
        sel = query_set[key][4]
        for value in values:

            def value_constraints(x, sel=sel, value=value):
                count = 0
                for idx in value:
                    count += x[idx]
                return count - sel * n_row

            query_constraints_list.append(value_constraints)
    return query_constraints_list

In [45]:
query_constraints_list = query_constraints(query_set, query_to_variable)

In [46]:
query_constraints_list

[<function __main__.query_constraints.<locals>.value_constraints(x, sel=0.03340003078343851, value=[154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.03340003078343851, value=[4])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.00015391719255040788, value=[147])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.00015391719255040788, value=[115])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.013544712944435893, value=[120])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.013544712944435893, value=[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.5704171155918116, value=[122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138])>,
 <function __main__.query_constraints.<locals>.value_constrai

In [47]:
def fun():
    def error(x):
        total = 0
        query_diff = np.array(query_constraints_list).reshape(-1, args.num_conditions)
        total += sum(
            [
                (diff[e[0]](x) - diff[e[1]](x)) ** 2
                for diff in query_diff
                for e in itertools.combinations(range(args.num_conditions), 2)
            ]
        )
        total += sum([constraint(x) ** 2 for constraint in query_constraints_list])
        return total / (n_row) ** 2

    return error

In [48]:
def constraints():
    # query_ineq_constraints = [{'type': 'ineq', 'fun': constraint} for constraint in query_constraints_list]
    table_length_constraints = [
        {"type": "eq", "fun": constraint} for constraint in table_constraints_list
    ]
    return table_length_constraints  # + query_ineq_constraints

In [49]:
def x0():
    x0 = []
    for value in column_to_variable.values():
        x0 += [n_row / len(value)] * len(value)
    return x0

In [50]:
res = optimize.minimize(
    fun(),
    x0(),
    method="SLSQP",
    constraints=constraints(),
    bounds=np.array([[0, None]] * total_variables),
    # tol=1e-323,
    options={"maxiter": 1e10},
)

KeyboardInterrupt: 

In [ ]:
print(res)

In [ ]:
def randomized_rouding(x):
    int_x = deepcopy(x)
    for i in range(len(x)):
        xi = x[i]
        floor = np.floor(xi)
        ceil = np.ceil(xi)
        if not floor == ceil:
            int_x[i] = np.random.choice([floor, ceil], p=[xi - floor, ceil - xi])
    return int_x

In [ ]:
int_x = randomized_rouding(res.x).astype(int)

In [ ]:
int_x

In [ ]:
fun()(int_x)

In [ ]:
query_error = [func(int_x) for func in query_constraints_list]

In [ ]:
query_error

In [ ]:
origin_column_length = [func(res.x) + n_row for func in table_constraints_list]
origin_column_length

In [ ]:
column_length = [func(int_x) + n_row for func in table_constraints_list]
column_length

In [ ]:
table_generate_length = max(column_length)

In [ ]:
table_generate_length

In [ ]:
# generate data

In [ ]:
def generate_table_data(column_to_variable, int_x, table_generate_length):
    col_list = []
    for k, v in column_to_variable.items():
        col_list.append([])
        for idx in v:
            exec(f"col_list[{k}] += [variable_to_interval[idx]] * int_x[idx]")
    for col in col_list:
        col += [0] * (table_generate_length - len(col))
        col.sort()
    col_df = {}
    for i in range(n_column):
        exec(f"col_df['col_{i}'] = col_list[i]")
    return pd.DataFrame(col_df)

In [ ]:
table_generate = generate_table_data(column_to_variable, int_x, table_generate_length)

In [ ]:
table_generate

In [ ]:
def Q_error(query_set, our_table):
    diff = []
    for query in query_set:
        sentence = ""
        for i in range(len(query[0])):
            if i != 0:
                sentence += " and "
            sentence += f"col_{query[1][i]}"
            if query[2][i][0] == "=":
                sentence += "=="
            else:
                sentence += query[2][i][0]
            sentence += f"{query[3][i][0]}"
        sel = our_table.query(sentence).shape[0]  # / our_table.shape[0]
        sel2 = query[4] * n_row
        if sel == 0:
            sel += 1
        if sel2 == 0:
            sel2 += 1
        if sel < sel2:
            diff.append(float(sel2) / sel)
        else:
            diff.append(float(sel) / sel2)
    return diff

In [ ]:
diff = Q_error(query_set, table_generate)

In [ ]:
print(f"min:    {np.min(diff)}")
print(f"10:     {np.percentile(diff, 10)}")
print(f"20:     {np.percentile(diff, 20)}")
print(f"30:     {np.percentile(diff, 30)}")
print(f"40:     {np.percentile(diff, 40)}")
print(f"median: {np.median(diff)}")
print(f"60:     {np.percentile(diff, 60)}")
print(f"70:     {np.percentile(diff, 70)}")
print(f"80:     {np.percentile(diff, 80)}")
print(f"mean:   {np.mean(diff)}")
print(f"90:     {np.percentile(diff, 90)}")
print(f"max:    {np.max(diff)}")

In [ ]:
sorted(diff)

In [ ]:
pd.DataFrame(diff).hist(figsize=(20, 8))

In [ ]:
# 查看各部分的selectivity

In [ ]:
sentence_list = []
for query in query_set:
    sentence = ""
    for i in range(len(query[0])):
        if i != 0:
            sentence += " and "
        sentence += f"col_{query[1][i]}"
        if query[2][i][0] == "=":
            sentence += "=="
        else:
            sentence += query[2][i][0]
        sentence += f"{query[3][i][0]}"
    sentence_list.append(sentence)

selectivity = []
for sentence in sentence_list:
    selectivity.append(table_generate.query(sentence).shape[0] / table_generate.shape[0])

In [ ]:
selectivity

In [79]:
# 创建一个表，所有单元格初始化为nan

In [80]:
df = pd.DataFrame(
    columns=[f"col_{i}" for i in range(n_column)], index=[i for i in range(table_generate_length)]
)

In [81]:
df

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
df["col_0"].notnull()

0        True
1       False
2       False
3       False
4       False
        ...  
6494    False
6495    False
6496    False
6497    False
6498    False
Name: col_0, Length: 6499, dtype: bool

In [281]:
common_available_idx = df[df["col_0"].notnull() == False].index.tolist()
first_idx = common_available_idx[0]

In [283]:
df[["col_0", "col_1"]].notnull() == False

,col_0,col_1
0,False,True
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
6494,True,True
6495,True,True
6496,True,True
6497,True,True


In [296]:
column11 = [0, 2]

In [302]:
exec("a = [df[df[f'col_{i}'].notnull() == False].index.tolist() for i in column11]")

In [264]:
df[df["col_0"].notnull() == False][["col_0", "col_1"]]

,col_0,col_1
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
...,...,...
6494,NaN,NaN
6495,NaN,NaN
6496,NaN,NaN
6497,NaN,NaN


In [64]:
query_to_variable

{0: [[154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165], [4]],
 1: [[147], [115]],
 2: [[120], [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]],
 3: [[122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138],
  [30, 31, 32, 33, 34, 35, 36, 37]],
 4: [[43, 44, 45, 46, 47, 48, 49, 50], [78]],
 5: [[24], [131, 132, 133, 134, 135, 136, 137, 138]],
 6: [[48, 49, 50], [75]],
 7: [[171], [47]],
 8: [[176], [102, 103, 104, 105, 106, 107]],
 9: [[171], [85, 86, 87, 88, 89, 90, 91, 92, 93, 94]],
 10: [[85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100],
  [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]],
 11: [[170], [30, 31, 32, 33]],
 12: [[108, 109, 110, 111, 112, 113, 114, 115, 116],
  [124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138]],
 13: [[159], [15]],
 14: [[73],
  [117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,

In [65]:
query_constraints_list

[<function __main__.query_constraints.<locals>.value_constraints(x, sel=0.03340003078343851, value=[154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.03340003078343851, value=[4])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.00015391719255040788, value=[147])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.00015391719255040788, value=[115])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.013544712944435893, value=[120])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.013544712944435893, value=[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83])>,
 <function __main__.query_constraints.<locals>.value_constraints(x, sel=0.5704171155918116, value=[122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138])>,
 <function __main__.query_constraints.<locals>.value_constrai

In [73]:
variable_length = [func(int_x, sel=0) for func in query_constraints_list]
# query_constraints_list[5](int_x, sel=0)

In [74]:
variable_length

[6506,
 465,
 2166,
 240,
 171,
 4649,
 2166,
 5797,
 360,
 6073,
 5917,
 6072,
 5768,
 240,
 464,
 465,
 3602,
 465,
 434,
 5698,
 1300,
 341,
 3903,
 2849,
 4477,
 1183,
 240,
 2161,
 2164,
 434,
 2728,
 296,
 1734,
 1860,
 434,
 296,
 3255,
 326,
 2522,
 360,
 4259,
 1299,
 296,
 2166,
 2168,
 1300,
 6138,
 2166,
 4735,
 2644,
 1440,
 6045,
 5525,
 5639,
 2166,
 1959,
 407,
 3255,
 407,
 2166,
 296,
 1628,
 2286,
 1299,
 240,
 866,
 434,
 4774,
 1440,
 6072,
 1299,
 296,
 3264,
 465,
 241,
 465,
 5204,
 1300,
 2166,
 171,
 434,
 241,
 3037,
 3410,
 4322,
 465,
 2166,
 3663,
 4084,
 2790,
 4650,
 6479,
 1300,
 407,
 2958,
 2046,
 296,
 240,
 3751,
 3069,
 240,
 1026,
 5206,
 465,
 1299,
 4433,
 2603,
 4259,
 3362,
 2644,
 1299,
 2166,
 3218,
 3551,
 930,
 2166,
 407,
 3470,
 3720,
 1202,
 4430,
 4682,
 241,
 2161,
 465,
 5325,
 465,
 5639,
 1299,
 341,
 296,
 4070,
 360,
 5287,
 241,
 2166,
 5829,
 1479,
 4196,
 240,
 1395,
 2601,
 2166,
 2882,
 1920,
 295,
 5402,
 296,
 4566,
 4650,

In [393]:
start = 0
for k, v in query_to_variable.items():
    common_available_idx = set(range(table_generate_length))
    column = query_list[k][0]
    for col in column:
        available_idx = df[df[f"col_{col}"].notnull() == False].index.tolist()
        common_available_idx = common_available_idx & set(available_idx)

    end = start + len(column)
    # 比较每个列里的最小的数量
    min_length = min(variable_length[start:end])
    # 拿到共同索引
    print(len(common_available_idx))
    # 判断共同索引空间是否足够装下所有的变量
    # 找到变量
    # 取左端点的值
    # 插入数据
    # 插入剩余的数据
    # 如果剩余的索引空间装不下所有的变量，按照索引空间有多大，就放多少的变量进去
    # 找到变量
    # 取左端点的值
    # 插入数据
    # 插入剩余的数据
    # 如果索引空间为0
    # 找到变量
    # 取左端点的值
    # 插入数据（在自己列的可用索引空间里找位置随便插）# 不好，维护一个每个变量还剩余多少值没有插，等到最后所有的都插完了之后，再插
    # 插入剩余的数据
    # first_idx = sorted(list(common_available_idx))[0]
    start = end
    # print(first_idx)

6497
[1, 2]
6498
[3, 4]
6499
[5, 6]


In [ ]:
# available_idx = [df[df[f'col_{i}'].notnull() == False].index.tolist() for i in column11]

In [266]:
used_x  # 如果按query插的话，应该用不到used_x

{1, 5, 6, 12, 13, 20, 21, 22, 25, 26}

In [267]:
col_used_x  # 如果按query插的话，应该用不到col_used_x

{0: [1],
 1: [],
 2: [5, 6],
 3: [],
 4: [],
 5: [12, 13],
 6: [],
 7: [],
 8: [20, 21, 22],
 9: [],
 10: [25, 26],
 11: [],
 12: []}

In [270]:
col_unused_x

{0: [0, 2],
 1: [3, 4],
 2: [7],
 3: [8, 9],
 4: [10, 11],
 5: [14],
 6: [15, 16],
 7: [17, 18],
 8: [19],
 9: [23, 24],
 10: [27],
 11: [28, 29],
 12: [30, 31]}

In [196]:
query_to_variable

{0: [[25, 26], [1]], 1: [[20], [12, 13]], 2: [[21, 22], [5, 6]]}

In [ ]:
# 插入query中涉及到的列
# 第一种方式，按query插，维护一个每列已经使用了的索引位置
# 第二种方式，按列插，维护一个每列中参与query的列表
# 第三种方式，按列-按query插，维护一个每列中参与query的列表，插入一个变量时，通过query找到和这个变量有关的其他变量，维护一个每列已经使用了的索引位置，找到两个变量共同的可使用索引范围，插入后更新所有列的可使用索引位置列表

In [ ]:
# 创建一个varible list，插入一个x后，列表（集合set实现）中的元素append x的序号，确认遍历了所有的 x，列表长度等于variable的数量

In [ ]:
# 插入 column_unused_variable 中的值

In [ ]:
column_unused_variable

In [ ]:
# 在最后检查表中是否有nan的值，sum列求和，最后蹦出一个ture就代表已经插入完毕

In [ ]:
# 以下为实验

In [28]:
list1 = [f"con{i}" for i in range(10)]
for func in list1:
    globals()[func] = []
A

[]

In [ ]:
# SLSQP，OSQP，pulp，

# Pulp

In [196]:
import pulp

In [ ]:
MyProbLP = pulp.LpProblem(
    "LPProbDemo1", sense=pulp.LpMaximize
)  # 定义问题，求最小值 pulp.LpMinimize / 最大值

In [204]:
x1 = pulp.LpVariable("x1", lowBound=0, upBound=7, cat="Integer")
x2 = pulp.LpVariable("x2", lowBound=0, upBound=7, cat="Integer")
x3 = pulp.LpVariable("x3", lowBound=0, upBound=7, cat="Continuous")

In [205]:
MyProbLP += 2 * x1 + 3 * x2 - 5 * x3  # 设置目标函数
MyProbLP += 2 * x1 - 5 * x2 + x3 >= 10  # 不等式约束
MyProbLP += x1 + 3 * x2 + x3 <= 12  # 不等式约束
MyProbLP += x1 + x2 + x3 == 7  # 等式约束

In [206]:
MyProbLP.solve()

1

In [210]:
print(MyProbLP)  # 输出问题设定参数和条件

LPProbDemo1:
MAXIMIZE
2*x1 + 3*x2 + -5*x3 + 0
SUBJECT TO
_C1: 2 x1 - 5 x2 + x3 >= 10

_C2: x1 + 3 x2 + x3 <= 12

_C3: x1 + x2 + x3 = 7

VARIABLES
0 <= x1 <= 7 Integer
0 <= x2 <= 7 Integer
x3 <= 7 Continuous



In [207]:
print("Status:", pulp.LpStatus[MyProbLP.status])  # 输出求解状态

Status: Optimal


In [208]:
for v in MyProbLP.variables():
    print(v.name, "=", v.varValue)  # 输出每个变量的最优值

x1 = 7.0
x2 = 0.0
x3 = 0.0


In [209]:
print("F(x) = ", pulp.value(MyProbLP.objective))  # 输出最优解的目标函数值

F(x) =  14.0


In [ ]:
# 第二个实例

In [211]:
import pulp  # 导入 pulp库= 关注 Youcans，分享原创系列 https://blog.csdn.net/youcans =

# 1. 建立问题
AlloyModel = pulp.LpProblem("钢材生产问题", pulp.LpMinimize)
# 2. 建立变量
material = ["废料1", "废料2", "废料3", "废料4", "镍", "铬", "钼"]
mass = pulp.LpVariable.dicts("原料", material, lowBound=0, cat="Continuous")
# 3. 设置目标函数
cost = {"废料1": 16, "废料2": 10, "废料3": 8, "废料4": 9, "镍": 48, "铬": 60, "钼": 53}
AlloyModel += pulp.lpSum([cost[item] * mass[item] for item in material]), "总生产成本"
# # 4. 施加约束
carbonPercent = {"废料1": 0.8, "废料2": 0.7, "废料3": 0.85, "废料4": 0.4, "镍": 0, "铬": 0, "钼": 0}
NiPercent = {"废料1": 18, "废料2": 3.2, "废料3": 0, "废料4": 0, "镍": 100, "铬": 0, "钼": 0}
CrPercent = {"废料1": 12, "废料2": 1.1, "废料3": 0, "废料4": 0, "镍": 0, "铬": 100, "钼": 0}
MoPercent = {"废料1": 0, "废料2": 0.1, "废料3": 0, "废料4": 0, "镍": 0, "铬": 0, "钼": 100}
AlloyModel += pulp.lpSum([mass[item] for item in material]) == 1000, "质量约束"
AlloyModel += (
    pulp.lpSum([carbonPercent[item] * mass[item] for item in material]) >= 0.65 * 1000,
    "碳最小占比",
)
AlloyModel += (
    pulp.lpSum([carbonPercent[item] * mass[item] for item in material]) <= 0.75 * 1000,
    "碳最大占比",
)
AlloyModel += (
    pulp.lpSum([NiPercent[item] * mass[item] for item in material]) >= 3.0 * 1000,
    "镍最小占比",
)
AlloyModel += (
    pulp.lpSum([NiPercent[item] * mass[item] for item in material]) <= 3.5 * 1000,
    "镍最大占比",
)
AlloyModel += (
    pulp.lpSum([CrPercent[item] * mass[item] for item in material]) >= 1.0 * 1000,
    "铬最小占比",
)
AlloyModel += (
    pulp.lpSum([CrPercent[item] * mass[item] for item in material]) <= 1.2 * 1000,
    "铬最大占比",
)
AlloyModel += (
    pulp.lpSum([MoPercent[item] * mass[item] for item in material]) >= 1.1 * 1000,
    "钼最小占比",
)
AlloyModel += (
    pulp.lpSum([MoPercent[item] * mass[item] for item in material]) <= 1.3 * 1000,
    "钼最大占比",
)
AlloyModel += mass["废料1"] <= 75, "废料1可用量"
AlloyModel += mass["废料2"] <= 250, "废料2可用量"
# 5. 求解
AlloyModel.solve()
# 6. 打印结果
print(AlloyModel)  # 输出问题设定参数和条件
print("优化状态:", pulp.LpStatus[AlloyModel.status])
for v in AlloyModel.variables():
    print(v.name, "=", v.varValue)
print("最优总成本 = ", pulp.value(AlloyModel.objective))

钢材生产问题:
MINIMIZE
16*原料_废料1 + 10*原料_废料2 + 8*原料_废料3 + 9*原料_废料4 + 53*原料_钼 + 60*原料_铬 + 48*原料_镍 + 0
SUBJECT TO
质量约束: 原料_废料1 + 原料_废料2 + 原料_废料3 + 原料_废料4 + 原料_钼 + 原料_铬 + 原料_镍 = 1000

碳最小占比: 0.8 原料_废料1 + 0.7 原料_废料2 + 0.85 原料_废料3 + 0.4 原料_废料4 >= 650

碳最大占比: 0.8 原料_废料1 + 0.7 原料_废料2 + 0.85 原料_废料3 + 0.4 原料_废料4 <= 750

镍最小占比: 18 原料_废料1 + 3.2 原料_废料2 + 100 原料_镍 >= 3000

镍最大占比: 18 原料_废料1 + 3.2 原料_废料2 + 100 原料_镍 <= 3500

铬最小占比: 12 原料_废料1 + 1.1 原料_废料2 + 100 原料_铬 >= 1000

铬最大占比: 12 原料_废料1 + 1.1 原料_废料2 + 100 原料_铬 <= 1200

钼最小占比: 0.1 原料_废料2 + 100 原料_钼 >= 1100

钼最大占比: 0.1 原料_废料2 + 100 原料_钼 <= 1300

废料1可用量: 原料_废料1 <= 75

废料2可用量: 原料_废料2 <= 250

VARIABLES
原料_废料1 Continuous
原料_废料2 Continuous
原料_废料3 Continuous
原料_废料4 Continuous
原料_钼 Continuous
原料_铬 Continuous
原料_镍 Continuous

优化状态: Optimal
原料_废料1 = 75.0
原料_废料2 = 90.909091
原料_废料3 = 672.28283
原料_废料4 = 137.30808
原料_钼 = 10.909091
原料_铬 = 0.0
原料_镍 = 13.590909
最优总成本 =  9953.671725000002


## many attributes

In [236]:
import sympy as sym

In [262]:
# 2. 建立变量
x1_vector = ["x1", "x2", "x3", "x4"]
x1 = pulp.LpVariable.dicts("x1", x1_vector, lowBound=0, cat="Continuous")

In [263]:
L = np.array([[1, 1, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1]])

In [264]:
x1 = sym.symbols("x1 x2 x3 x4")

In [265]:
error = L @ x1 - b

In [266]:
error[0]

x1 + x2 + x3 + x4 - 50

In [233]:
import pulp

In [267]:
# 1. 建立最小化问题
AlloyModel = pulp.LpProblem("many_attributes", pulp.LpMinimize)

In [268]:
x1 = pulp.LpVariable("x1", lowBound=0, upBound=7, cat="Integer")
x2 = pulp.LpVariable("x2", lowBound=0, upBound=7, cat="Integer")
x3 = pulp.LpVariable("x3", lowBound=0, upBound=7, cat="Continuous")
x4 = pulp.LpVariable("x4", lowBound=0, upBound=7, cat="Continuous")

In [269]:
# 3. 设置目标函数
L = np.array([[1, 1, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1]])
b = np.array([50, 30, 40])


AlloyModel += x1 + x2 + x3 + x4  # pulp.lpSum([item**2 for item in x1]), "总生产成本"

In [ ]:
# # 4. 施加约束
carbonPercent = {"废料1": 0.8, "废料2": 0.7, "废料3": 0.85, "废料4": 0.4, "镍": 0, "铬": 0, "钼": 0}
NiPercent = {"废料1": 18, "废料2": 3.2, "废料3": 0, "废料4": 0, "镍": 100, "铬": 0, "钼": 0}
CrPercent = {"废料1": 12, "废料2": 1.1, "废料3": 0, "废料4": 0, "镍": 0, "铬": 100, "钼": 0}
MoPercent = {"废料1": 0, "废料2": 0.1, "废料3": 0, "废料4": 0, "镍": 0, "铬": 0, "钼": 100}
AlloyModel += pulp.lpSum([mass[item] for item in material]) == 1000, "质量约束"
AlloyModel += (
    pulp.lpSum([carbonPercent[item] * mass[item] for item in material]) >= 0.65 * 1000,
    "碳最小占比",
)
AlloyModel += (
    pulp.lpSum([carbonPercent[item] * mass[item] for item in material]) <= 0.75 * 1000,
    "碳最大占比",
)
AlloyModel += (
    pulp.lpSum([NiPercent[item] * mass[item] for item in material]) >= 3.0 * 1000,
    "镍最小占比",
)
AlloyModel += (
    pulp.lpSum([NiPercent[item] * mass[item] for item in material]) <= 3.5 * 1000,
    "镍最大占比",
)
AlloyModel += (
    pulp.lpSum([CrPercent[item] * mass[item] for item in material]) >= 1.0 * 1000,
    "铬最小占比",
)
AlloyModel += (
    pulp.lpSum([CrPercent[item] * mass[item] for item in material]) <= 1.2 * 1000,
    "铬最大占比",
)
AlloyModel += (
    pulp.lpSum([MoPercent[item] * mass[item] for item in material]) >= 1.1 * 1000,
    "钼最小占比",
)
AlloyModel += (
    pulp.lpSum([MoPercent[item] * mass[item] for item in material]) <= 1.3 * 1000,
    "钼最大占比",
)
AlloyModel += mass["废料1"] <= 75, "废料1可用量"
AlloyModel += mass["废料2"] <= 250, "废料2可用量"

In [270]:
# 5. 求解
AlloyModel.solve()

1

In [271]:
# 6. 打印结果
print(AlloyModel)  # 输出问题设定参数和条件

many_attributes:
MINIMIZE
1*x1 + 1*x2 + 1*x3 + 1*x4 + 0
VARIABLES
0 <= x1 <= 7 Integer
0 <= x2 <= 7 Integer
x3 <= 7 Continuous
x4 <= 7 Continuous



In [272]:
print("优化状态:", pulp.LpStatus[AlloyModel.status])
for v in AlloyModel.variables():
    print(v.name, "=", v.varValue)
print("最优总成本 = ", pulp.value(AlloyModel.objective))

优化状态: Optimal
x1 = 0.0
x2 = 0.0
x3 = 0.0
x4 = 0.0
最优总成本 =  0.0


In [59]:
def constraints():
    return (
        {"type": "eq", "fun": lambda x: x[0] + x[1] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[2] - n_row},
        {"type": "eq", "fun": lambda x: x[3] + x[4] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[5] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[6] - n_row},
        {"type": "eq", "fun": lambda x: x[7] + x[8] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[9] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[10] - n_row},
        {"type": "eq", "fun": lambda x: x[11] + x[12] + x[13] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[14] - n_row},
        {"type": "eq", "fun": lambda x: x[15] + x[16] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[17] - n_row},
        # {'type': 'eq', 'fun': lambda x: x[18] - n_row},
    )